# 🪄 Defining types

In [1]:
from typing import List, Dict

# imports
from queue import PriorityQueue

# 🌲 Tree Class

In [2]:
class Tree:
  def __init__(self, name: str):
      """Initialize a tree with a name and an empty graph."""
      self.__name = name
      self.__graph: Dict[int, Dict[int, int]] = {}

  def _validate_vertex(self, v: int) -> None:
      """Check if vertex exists in the graph."""
      if v not in self.__graph:
          raise ValueError(f"🔖 Error: Vertex {v} not found")

  def add_edge(self, v: int, u: int, w: int) -> None:
      """Add an edge between two vertices with a weight."""
      if v not in self.__graph:
          self.__graph[v] = {}
      if u not in self.__graph:
          self.__graph[u] = {}

      self.__graph[v][u] = w
      self.__graph[u][v] = w

  def add_tree(self, tree: List[List[int]]) -> None:
      """Add a tree to the graph."""
      for v, u, w in tree:
          self.add_edge(v, u, w)

  # def update_edge(self, v: int, u: int, w: int) -> None:
  #     """Update the weight of an edge between two vertices."""
  #     if v in self.__graph and u in self.__graph:
  #         self.__graph[v][u] = w
  #         self.__graph[u][v] = w

  # def del_edge(self, v: int, u: int) -> None:
  #     """Delete an edge between two vertices."""
  #     try:
  #         if v not in self.__graph or u not in self.__graph:
  #             raise ValueError(f"🔖 Error: Vertex {v if v not in self.__graph else u} not found")

  #         if u not in self.__graph[v] or v not in self.__graph[u]:
  #             raise ValueError(f"🔖 Error: Edge ({v}, {u}) not found")

  #         del self.__graph[v][u]
  #         del self.__graph[u][v]
  #     except ValueError as e:
  #         print(e)

  # def clear_graph(self) -> None:
  #     """Clear the graph of all vertices and edges."""
  #     self.__graph.clear()

  def display_graph(self) -> None:
      """Display the graph with vertices and edges."""
      for v, edges in self.__graph.items():
          print(f"{v}: {[{u: w} for u, w in edges.items()]}\n")

  def bfs(self, start: int, goal: int) -> tuple[List[int], int]:
      """BFS traversal from start to goal vertex."""
      try:
          self._validate_vertex(start)
          self._validate_vertex(goal)

          if start == goal:
              return [start], 0

          visited = set()
          queue = [(start, [start], 0)]

          while queue:
              vertex, path, cost = queue.pop(0)
              visited.add(vertex)

              for neighbor, weight in self.__graph[vertex].items():
                  if neighbor == goal:
                      return path + [neighbor], cost + weight
                  elif neighbor not in visited:
                      queue.append((neighbor, path + [neighbor], cost + weight))

          return [], 0

      except ValueError as e:
          print(e)
          return [], 0

  def dfs(self, start: int, goal: int) -> tuple[List[int], int]:
      """DFS traversal from start to goal vertex."""
      try:
          self._validate_vertex(start)
          self._validate_vertex(goal)

          visited = set()
          path = []

          def dfs_recursive(vertex: int, cost: int) -> tuple[bool, int]:
              visited.add(vertex)
              path.append(vertex)

              if vertex == goal:
                  return True, cost

              for neighbor, weight in self.__graph[vertex].items():
                  if neighbor not in visited:
                      found, total_cost = dfs_recursive(neighbor, cost + weight)
                      if found:
                          return True, total_cost

              path.pop()
              visited.remove(vertex)
              return False, 0

          found, cost = dfs_recursive(start, 0)
          return (path, cost) if found else ([], 0)

      except ValueError as e:
          print(e)
          return [], 0

  def ucs(self, start: int, goal: int) -> tuple[List[int], int]:
      """Uniform Cost Search from start to goal vertex using PriorityQueue."""
      try:
          self._validate_vertex(start)
          self._validate_vertex(goal)

          if start == goal:
              return [start], 0

          visited = set()
          pq = PriorityQueue()
          pq.put((0, start, [start]))  # (cost, vertex, path)

          while not pq.empty():
              cost, vertex, path = pq.get()   # get() gets the lowest cost

              if vertex == goal:
                  return path, cost

              if vertex not in visited:
                  visited.add(vertex)
                  for neighbor, weight in self.__graph[vertex].items():
                      if neighbor not in visited:
                          pq.put((cost + weight, neighbor, path + [neighbor]))

          return [], 0

      except ValueError as e:
          print(e)
          return [], 0

  def dls(self, start: int, goal: int, depth: int) -> tuple[List[int], int]:
      """Depth Limited Search from start to goal vertex."""
      try:
          self._validate_vertex(start)
          self._validate_vertex(goal)

          def dls_recursive(vertex: int, depth: int, cost: int, path: List[int]) -> tuple[bool, List[int], int]:
              if vertex == goal:
                  return True, path, cost
              if depth > 0:
                  for neighbor, weight in self.__graph[vertex].items():
                      found, new_path, new_cost = dls_recursive(neighbor, depth - 1, cost + weight, path + [neighbor])
                      if found:
                          return True, new_path, new_cost
              return False, [], 0

          found, path, cost = dls_recursive(start, depth, 0, [start])
          return (path, cost) if found else ([], 0)

      except ValueError as e:
          print(e)
          return [], 0

  def ids(self, start: int, goal: int) -> tuple[List[int], int]:
    """Iterative Deepening Search from start to goal vertex."""
    try:
        self._validate_vertex(start)
        self._validate_vertex(goal)

        if start == goal:
            return [start], 0

        max_depth = len(self.__graph) - 1
        visited = set()
        frontier = [(start, [start], 0)]  # Initial frontier with start(s) node

        for _ in range(max_depth):
            next_frontier = []  # Store next level nodes (a,b,c)

            while frontier:
                vertex, path, cost = frontier.pop(0)

                if vertex == goal:
                    return path, cost

                if vertex not in visited:
                    visited.add(vertex)
                    # Add unvisited neighbors to next frontier
                    for neighbor, weight in self.__graph[vertex].items():
                        if neighbor not in visited:
                            next_frontier.append((
                                neighbor,
                                path + [neighbor],
                                cost + weight
                            ))

            frontier = next_frontier
            if not frontier:  # If no more nodes to explore
                break

        return [], 0

    except ValueError as e:
        print(e)
        return [], 0

  def __str__(self) -> str:
        ret = f"🌲Tree: {self.__name}\n📉Graph:\n"
        for v, edges in self.__graph.items():
            ret += f"{v}: {[{u: w} for u, w in edges.items()]}\n"
        return ret

# ⚡ Main Function

In [3]:
if __name__ == '__main__':
  # tree = [vertex1, vertex2, weightFrom1To2]
  tree = [
      ['A', 'B', 10],
      ['A', 'C', 15],
      ['B', 'D', 12],
      ['B', 'F', 15],
      ['D', 'F', 1],
      ['D', 'E', 2],
      ['C', 'E', 10],
      ['F', 'E', 5],
  ]

  tree2 = [
      ['S', 'A', 1],
      ['S', 'B', 2],
      ['S', 'C', 3],
      ['A', 'D', 2],
      ['A', 'E', 3],
      ['B', 'G', 2],
      ['C', 'F', 4],
      ['E', 'G', 1],
      ['D', 'H', 9],
  ]

treeA = Tree('CT_QUS')
treeA.add_tree(tree2)
print(treeA)

# path, cost = treeA.bfs(1, 5)
# print(f"\nBFS lowest weight path from 1 to 5: {path}, Cost: {cost}")

# path, cost = treeA.ids('A', 'E')
# print(f"\nIDS path from A to E: {path}, Cost: {cost}")

start, goal = 'S', 'G'

path, cost = treeA.dfs(start, goal)
print(f"\nDFS path from {start} to {goal}: {path} | Cost: {cost}")

path, cost = treeA.ucs(start, goal)
print(f"\nUCS path from {start} to {goal}: {path} | Cost: {cost}")


🌲Tree: CT_QUS
📉Graph:
S: [{'A': 1}, {'B': 2}, {'C': 3}]
A: [{'S': 1}, {'D': 2}, {'E': 3}]
B: [{'S': 2}, {'G': 2}]
C: [{'S': 3}, {'F': 4}]
D: [{'A': 2}, {'H': 9}]
E: [{'A': 3}, {'G': 1}]
G: [{'B': 2}, {'E': 1}]
F: [{'C': 4}]
H: [{'D': 9}]


DFS path from S to G: ['S', 'A', 'E', 'G'] | Cost: 5

UCS path from S to G: ['S', 'B', 'G'] | Cost: 4
